In [1]:
!uv pip install boto3
!uv pip install sagemaker

Using Python 3.10.15 environment at C:\Users\bored\Music\SagemekerLearning\sagelearner
Audited 1 package in 16ms
Using Python 3.10.15 environment at C:\Users\bored\Music\SagemekerLearning\sagelearner
Audited 1 package in 34ms


In [2]:
!uv pip install ipywidgets

Using Python 3.10.15 environment at C:\Users\bored\Music\SagemekerLearning\sagelearner
Audited 1 package in 7ms


In [3]:
import json
import pandas as pd
import logging
from ipywidgets import Dropdown

In [4]:
import boto3
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\bored\AppData\Local\sagemaker\sagemaker\config.yaml


In [5]:
sess = sagemaker.Session()
region = sess.boto_region_name
print(region)

us-east-1


In [6]:
# download JumpStart model_manifest file.
boto3.client("s3").download_file(
    f"jumpstart-cache-prod-{region}", "models_manifest.json", "models_manifest.json"
)
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

print("number of models: ", len(model_list))

number of models:  17074


In [7]:
model_df = pd.DataFrame(model_list)
model_df.sample(20)

,model_id,version,min_version,spec_key,provider,search_keywords
12244,tensorflow-ic-imagenet-mobilenet-v1-100-128-cl...,4.0.14,2.189.0,community_models/tensorflow-ic-imagenet-mobile...,tensorflow,NaN
14272,tensorflow-icembedding-imagenet-mobilenet-v1-1...,3.0.13,2.189.0,community_models/tensorflow-icembedding-imagen...,tensorflow,"[Vision, Image Embedding]"
14200,tensorflow-icembedding-imagenet-mobilenet-v1-1...,1.0.5,2.189.0,community_models/tensorflow-icembedding-imagen...,NaN,NaN
6181,huggingface-txt2img-sd-dreambooth-library-orig...,1.0.6,2.189.0,community_models/huggingface-txt2img-sd-dreamb...,NaN,NaN
10044,pytorch-tabtransformerclassification-model,2.0.15,2.189.0,community_models/pytorch-tabtransformerclassif...,pytorch,NaN
13970,tensorflow-icembedding-imagenet-mobilenet-v1-0...,1.0.4,2.189.0,community_models/tensorflow-icembedding-imagen...,NaN,NaN
13154,tensorflow-ic-swin-base-patch4-window12-384,1.0.1,2.80.0,community_models/tensorflow-ic-swin-base-patch...,NaN,NaN
10390,tensorflow-ic-bit-m-r152x4-imagenet21k,2.0.15,2.189.0,community_models/tensorflow-ic-bit-m-r152x4-im...,tensorflow,"[Vision, Image Classification]"
10720,tensorflow-ic-cait-s24-384,2.0.4,2.189.0,community_models/tensorflow-ic-cait-s24-384/sp...,NaN,NaN
16989,tensorflow-tcembedding-universal-sentence-enco...,2.0.9,2.189.0,community_models/tensorflow-tcembedding-univer...,tensorflow,NaN


In [8]:
od_models = []
for model in model_list:
    model_id = model["model_id"]
    if ("-od-" in model_id or "-od1-" or "-od2-"in model_id) and model_id not in od_models:
        od_models.append(model_id)

print(f"Number of od models available for inference: {len(od_models)}")

Number of od models available for inference: 999


In [9]:
infer_model_dropdown = Dropdown(
    options=od_models,
    value="pytorch-od-nvidia-ssd",
    description="Select a model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

In [10]:
display(infer_model_dropdown)

Dropdown(description='Select a model:', index=632, layout=Layout(width='max-content'), options=('autogluon-cla…

In [11]:
# filter-out all the Image Classification models from the manifest list.
ic_models = []
for model in model_list:
    model_id = model["model_id"]
    if ("-ic-" in model_id) and model_id not in ic_models:
        ic_models.append(model_id)

print(f"Number of ic models available for inference: {len(ic_models)}")

Number of ic models available for inference: 162


In [ ]:

infer_model_dropdown = Dropdown(
    options=ic_models,
    value="pytorch-ic-alexnet",
    description="Select a model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

In [13]:
display(infer_model_dropdown)

Dropdown(description='Select a model:', layout=Layout(width='max-content'), options=('pytorch-ic-alexnet', 'py…

In [14]:
print(infer_model_dropdown.value)

pytorch-ic-mobilenet-v2
